In [1]:
import requests
import json
from PIL import Image
from io import BytesIO
import re
import demjson
import pandas as pd
from pandas import DataFrame
from datetime import datetime, timedelta , date
import numpy as np
import ast
from pandas.io.json import json_normalize

In [2]:
# r = requests.post("http://192.168.1.30:88/SYSA/mobilephone/getyzm.asp?serialnum=616EA64CD6A65AA")
# print(r.status_code, r.reason)
# i = Image.open(BytesIO(r.content))

In [3]:
rowsPerPage = 100000

today = date.today()

yesterday = date.today() - timedelta(1)
yesterday = yesterday.strftime('%Y-%m-%d')

start_date = "2018-05-26"

start_date, end_date = yesterday, yesterday

In [5]:
rndcode = ""
user = "***"
password = "***"
serialnum = "***"

data ='{session:"******",datas:[{id:"user",val:"txt:%s"},{id:"password",val:"txt:%s"},{id:"serialnum",val:"%s"},{id:"rndcode",val:"%s"}]}'%(user,password,serialnum,rndcode)
        
headers = {"Content-Type":"application/zsml;charset=gb2312"}
res = requests.post("http://192.168.1.30:88/SYSA/mobilephone/login.asp",data=data,headers=headers)
# print(res.status_code)
ses_text = res.text
ses_text

'{header:{status:0,message:"OK",language:"cn",session:"e482239ed44caafa631b160516537b128e9263ad55408f28b693374a15d6726a8597f6688405b9885ebba113e7e978a235e05f6f36ff3f44"},body:{action:"sys",model:"init",sys:{info:{floatbit:2,moneybit:2,disbit:2,dismax:1,hlbit:2,mxbit:500,remind:300,isopenrecord:false,ratebit:300,uniqueName:"cdb90870289093b61cc89ae72f786c74"},trees:{nodes:[{id:"SystemManage",action:"_none",text:"系统操作",nodes:[{id:"about",ico:"about",action:"_client",text:"关于",url:"100103",nodes:[],deep:0}],deep:0}],deep:0},homeurl:"mobilephone/home2.asp",exiturl:"mobilephone/logout.asp",helpurl:"mobilephone/cshelp/index.html",adseturl:"mobilephone/systemmanage/setadsearchfields.asp",uploadsmsurl:"mobilephone/uploadsms.asp",uploadcallurl:"mobilephone/uploadcall.asp?mode=mobile",gpssaveurl:"mobilephone/salesmanage/gps/add.asp",uploadgpsmap:"mobilephone/upload.asp?userid={@session}",data:"{\\"dhmanage\\":1,\\"savephone\\":0}"}}}'

In [6]:
t = re.search('(?:session:)(.*)(?=},body)',ses_text).group()[9:-1]
t

'e482239ed44caafa631b160516537b128e9263ad55408f28b693374a15d6726a8597f6688405b9885ebba113e7e978a235e05f6f36ff3f44'

In [7]:
headers = {"Content-Type":"application/zsml;charset=gb2312"}
data = '{session:"%s",\
        cmdkey:"refresh",\
        datas:[{id:"pagesize",val:"%s"},\
        {id:"dateQD_0",val:"%s"},\
        {id:"dateQD_1",val:"%s"}\
        ]}'%(t,rowsPerPage,start_date,end_date)
# data

In [8]:
%%time
session = requests.Session()
s = session.post("http://192.168.1.30:88/SYSA/mobilephone/salesmanage/contract/billlist.asp",data=data,headers=headers)
org_text = s.text

Wall time: 593 ms


In [18]:
type(org_text)

str

In [12]:
# org_text

In [16]:
%%time
json_obj = demjson.decode(org_text)
# print(json_obj)

Wall time: 107 ms


In [17]:
type(json_obj)

dict

In [20]:
col_dict = json_obj['body']['source']['table']['cols']
# col_dict

In [23]:
col_name = []
for i in col_dict:
    c = i['id']
    col_name.append(c)
# col_name

In [25]:
row_dict = json_obj['body']['source']['table']['rows']
# print(row_dict)

In [26]:
contract_list = pd.DataFrame(row_dict, columns = col_name)
contract_list.shape

(33, 16)

In [66]:
# contract_list.to_csv('contract_list.csv',encoding = 'utf-8',index=False)

In [27]:
contract_list.head(2)

,ord,title,xsry,khmc,moneyall,hkmoney,ckjz,fhjz,dateQD,dateKS,dateZZ,htbz,htfl,spMan,spzt,url
0,41523,（成交）宝达集团有限公司/江苏宝达汽车股份有限公司HST20180612050,聂志华,（成交）宝达集团有限公司/江苏宝达汽车股份有限公司,"2,400.00",0.00,出库完毕,发货完毕,2018-06-12,2018-06-12,2019-06-11,RMB,视频会议软件产品销售,,审批通过,_url
1,41522,北京巅峰智业旅游文化创意股份有限公司HST20180612049,马一帆,北京巅峰智业旅游文化创意股份有限公司,0.00,0.00,出库完毕,发货完毕,2018-06-12,2018-06-12,2019-06-11,RMB,硬件配件销售,,审批通过,_url


In [29]:
# contract_list

In [30]:
contract_list.ord = contract_list.ord.astype(int)
contract_list.ord.describe()

count       33.000000
mean     41490.272727
std         17.929154
min      41466.000000
25%      41475.000000
50%      41486.000000
75%      41505.000000
max      41523.000000
Name: ord, dtype: float64

# contract details

In [31]:
for i in range(contract_list.ord.min(), contract_list.ord.max()):
    print(i)

41466
41467
41468
41469
41470
41471
41472
41473
41474
41475
41476
41477
41478
41479
41480
41481
41482
41483
41484
41485
41486
41487
41488
41489
41490
41491
41492
41493
41494
41495
41496
41497
41498
41499
41500
41501
41502
41503
41504
41505
41506
41507
41508
41509
41510
41511
41512
41513
41514
41515
41516
41517
41518
41519
41520
41521
41522


In [32]:
%%time
json_contract_detail_list = []
for i in range(contract_list.ord.min(), contract_list.ord.max()):
    ord_num = i
    headers = {"Content-Type":"application/zsml;charset=gb2312"}
    data = '{session:"%s",datas:[{id:"ord",val:"%s"}]}'%(t,ord_num)
    s = session.post("http://192.168.1.30:88/SYSA/mobilephone/salesmanage/contract/billservice.asp",data=data,headers=headers)
    contract_detail = s.text
    json_contract_detail = demjson.decode(contract_detail)
#     print(json_contract_detail['body']['model'])
    if json_contract_detail['body']['model'] == "message":
        None
    else:
        json_contract_detail_list.append(json_contract_detail)
    # contract_detail

Wall time: 1min 17s


In [33]:
len(json_contract_detail_list)

46

In [36]:
json_contract_detail

{'body': {'action': 'init',
  'bill': {'caption': '合同详情',
   'groups': [{'caption': '基本信息',
     'fields': [{'canset': False,
       'caption': '合同主题',
       'dbtype': 'string',
       'edit': False,
       'id': 'title',
       'maxl': 50,
       'minl': 0,
       'notnull': True,
       'onlyread': False,
       'post': '1',
       'source': {},
       'text': '北京巅峰智业旅游文化创意股份有限公司HST20180612049',
       'type': 'text',
       'visible': True},
      {'canset': False,
       'caption': '销售人员',
       'dbtype': 'string',
       'edit': False,
       'id': 'salesMan',
       'maxl': 50,
       'minl': 0,
       'notnull': True,
       'onlyread': False,
       'post': '1',
       'source': {},
       'text': '马一帆',
       'type': 'text',
       'visible': True},
      {'canset': False,
       'caption': '合同编号',
       'dbtype': 'string',
       'edit': False,
       'id': 'htid',
       'maxl': 50,
       'minl': 0,
       'notnull': True,
       'onlyread': False,
       'post': '1',
 

In [38]:
def make_row(json_object):
    table_dicts = {}
    for item in json_object['body']['bill']['groups']:
        for field in item['fields']:
            if set(("text", "caption")) <= set(field):
                table_dicts[field['caption']]=field['text']
            elif 'text' not in field:
                if 'caption' not in field:
                    None
                else:
                    table_dicts[field['caption']]=None
    return table_dicts

In [39]:
j_list = []
for json_contract_detail in json_contract_detail_list:
    dict_data = make_row(json_contract_detail)
    columns = list(dict_data.keys())
    values = list(dict_data.values())
    arr_len = len(values)
    df_temp = pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
    j_list.append(df_temp)
# j_list

In [40]:
len(j_list)

46

In [42]:
dfs = pd.concat(j_list,ignore_index = True)
# dfs

In [43]:
dfs.to_csv('CRM_API_当日合同明细1.csv',encoding = 'gbk')